<a href="https://colab.research.google.com/github/nataliepham6720/16-745_Optimal_Control/blob/main/cartpole_ilc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import block_diag
from scipy.sparse import kron, eye, block_diag as sp_block_diag, diags, vstack, hstack, csc_matrix
import osqp
from autograd import jacobian

# Define Cartpole Dynamics (basic placeholder)
class Cartpole:
    def __init__(self):
        self.mc = 1.0
        self.mp = 0.1
        self.l = 0.5
        self.g = 9.81

    def dynamics(self, x, u):
        mc, mp, l, g = self.mc, self.mp, self.l, self.g
        q = x[:2]
        qd = x[2:]
        s, c = np.sin(q[1]), np.cos(q[1])

        H = np.array([[mc + mp, mp * l * c], [mp * l * c, mp * l ** 2]])
        C = np.array([[0, -mp * qd[1] * l * s], [0, 0]])
        G = np.array([0, mp * g * l * s])
        B = np.array([1, 0])
        qdd = np.linalg.solve(H, -C @ qd - G + B * u[0])
        return np.concatenate([qd, qdd])

model = Cartpole()
n, m = 4, 1
N = 101
Tf = 5.0
h = Tf / (N - 1)

x0 = np.zeros(n)
xf = np.array([0, np.pi, 0, 0])

# Nominal LQR design
Q = np.diag([1.0]*n)
Qf = 100 * np.eye(n)
R = 0.1 * np.eye(m)

xnom = np.zeros((n, N))
unom = np.zeros((m, N - 1))

# Simulate nominal trajectory with zeros
for k in range(N-1):
    xnom[:,k+1] = xnom[:,k] + h * model.dynamics(xnom[:,k], np.zeros(m))

# Compute linearized dynamics
A = np.zeros((n, n, N - 1))
B = np.zeros((n, m, N - 1))

def rk4(f, x, u):
    f1 = f(x, u)
    f2 = f(x + 0.5 * h * f1, u)
    f3 = f(x + 0.5 * h * f2, u)
    f4 = f(x + h * f3, u)
    return x + (h / 6.0) * (f1 + 2*f2 + 2*f3 + f4)

for k in range(N - 1):
    A[:, :, k] = jacobian(lambda x: rk4(model.dynamics, x, unom[:,k]))(xnom[:,k])
    B[:, :, k] = jacobian(lambda u: rk4(model.dynamics, xnom[:,k], u))(unom[:,k].reshape(-1))

# LQR Gain computation
P = np.zeros((n, n, N))
K = np.zeros((m, n, N - 1))
P[:, :, -1] = Qf
for k in range(N - 2, -1, -1):
    BT_PB = B[:, :, k].T @ P[:, :, k + 1] @ B[:, :, k]
    BT_PA = B[:, :, k].T @ P[:, :, k + 1] @ A[:, :, k]
    K[:, :, k] = np.linalg.solve(R + BT_PB, BT_PA)
    A_BK = A[:, :, k] - B[:, :, k] @ K[:, :, k]
    P[:, :, k] = Q + K[:, :, k].T @ R @ K[:, :, k] + A_BK.T @ P[:, :, k + 1] @ A_BK

# Simulate true dynamics with feedback
xtraj = np.zeros((n, N))

for k in range(N - 1):
    unom[:,k] = -K[:, :, k] @ (xtraj[:,k] - xnom[:,k])
    xtraj[:,k+1] = rk4(model.dynamics, xtraj[:,k], unom[:,k])

# ILC QP setup
Nh = N
Qilc = csc_matrix(np.diag([0.01, 0, 1.0, 0]))
Rilc = csc_matrix(np.diag([0.1]))
Qf_sparse = csc_matrix(Qf)

blk_list = [sp_block_diag([Rilc, Qilc])]*(Nh-2) + [sp_block_diag([Rilc, Qf_sparse])]
H = sp_block_diag(blk_list)

q = np.zeros((n + m) * (Nh - 1))
for k in range(Nh - 2):
    err = xtraj[:, k+1] - xnom[:, k+1]
    q[k*(n+m)+m:k*(n+m)+m+n] = Qilc @ err
q[(Nh-2)*(n+m)+m:] = Qf @ (xtraj[:,Nh-1] - xnom[:,Nh-1])

# Constraint matrices
U = kron(eye(Nh - 1), np.hstack([np.eye(m), np.zeros((m, n))]))
D_blocks = []
for k in range(Nh - 1):
    row = np.zeros(((n, (n+m)*(Nh-1))))
    if k == 0:
        row[:, :m] = B[:, :, 0]
        row[:, m:m+n] = -np.eye(n)
    else:
        idx = k*(n+m)
        row[:, idx:idx+m] = B[:, :, k]
        row[:, idx+m:idx+m+n] = -np.eye(n)
        row[:, idx-(n+m)+m:idx-(n+m)+m+n] = A[:, :, k]
    D_blocks.append(row)
D = csc_matrix(np.vstack(D_blocks))

# Bounds
u_bnd = 5.0
lb = np.hstack([np.zeros(n*(Nh-1)), -u_bnd - unom.flatten()])
ub = np.hstack([np.zeros(n*(Nh-1)),  u_bnd - unom.flatten()])

# OSQP QP solve
qp = osqp.OSQP()
qp.setup(P=H, q=q, A=vstack([D, U]), l=lb, u=ub, verbose=False, eps_abs=1e-6, eps_rel=1e-6)
res = qp.solve()

ztraj = res.x
Δu = U @ ztraj
unom += Δu.reshape((m, Nh-1))
